# Step 2: Calibrated Street Network Generation

**Generate 20 new pedestrian networks calibrated to reference city metrics**

This notebook:
1. Loads reference metrics from Step 1
2. Generates 20 pedestrian networks matching segment length distributions
3. Adjusts metrics for doubled roads (MultiDiGraph)
4. Visualizes all 20 networks in one SVG

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Patch
from pathlib import Path
from collections import Counter
import math
import pickle
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
from shapely.ops import unary_union

%matplotlib inline
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10

print("✓ Libraries loaded")

## Configuration

In [ ]:
WINDOW_SIZE_M = 500  # 500×500m window
MIN_SEGMENT_LENGTH = 5.0  # Filter segments < 5m
NUM_NETWORKS_TO_GENERATE = 20  # Generate 20 networks

# Create output directories
Path("outputs/generated").mkdir(parents=True, exist_ok=True)
Path("outputs/generated/visualizations").mkdir(parents=True, exist_ok=True)
Path("outputs/generated/networks").mkdir(parents=True, exist_ok=True)

print(f"Window size: {WINDOW_SIZE_M}m × {WINDOW_SIZE_M}m")
print(f"Min segment length: {MIN_SEGMENT_LENGTH}m")
print(f"Networks to generate: {NUM_NETWORKS_TO_GENERATE}")
print("✓ Output directories created")

## Load Reference Data from Step 1

In [ ]:
# Load reference city data
with open('outputs/data/reference_cities_data.pkl', 'rb') as f:
    reference_data = pickle.load(f)

print("✓ Loaded reference data from Step 1")
print(f"\nReference cities: {list(reference_data.keys())}")

# Print summary with ADJUSTED metrics (divide by 2 for doubled metrics)
print("\n" + "="*70)
print("REFERENCE METRICS SUMMARY (ADJUSTED FOR DOUBLED ROADS)")
print("="*70)
print("\nMETRICS DIVIDED BY 2: Nodes, Edges, Node Density, Avg Degree")
print("METRICS NOT DOUBLED: Avg Segment Length, Orientation, Intelligibility")
print("="*70)

for city_key, data in reference_data.items():
    # Adjusted metrics (divided by 2)
    nodes_adj = data['graph'].number_of_nodes() // 2
    edges_adj = data['graph'].number_of_edges() // 2
    node_density_adj = data['morphology']['node_density'] / 2
    avg_degree_adj = data['morphology']['avg_degree'] / 2
    
    # Not doubled metrics (keep as-is)
    avg_seg_length = data['morphology']['avg_segment_length']
    intelligibility = data['syntax']['intelligibility']
    
    print(f"\n{city_key.upper()}:")
    print(f"  Network: {nodes_adj} nodes, {edges_adj} edges (adjusted)")
    print(f"  Node density: {node_density_adj:.1f} nodes/km² (adjusted)")
    print(f"  Avg degree: {avg_degree_adj:.2f} (adjusted)")
    print(f"  Avg segment length: {avg_seg_length:.1f}m (NOT doubled)")
    print(f"  Segment length samples: {len(data['morphology']['segment_lengths'])}")
    print(f"  Buildings: {len(data['buildings'])} (coverage: {data['building_metrics']['building_coverage_ratio']:.3f})")
    print(f"  Intelligibility: {intelligibility:.3f} (NOT doubled)")

## Network Generation Functions

In [ ]:
def sample_segment_length(reference_lengths):
    """
    Sample a segment length from reference distribution.
    
    Args:
        reference_lengths: List of segment lengths from reference city
    
    Returns:
        Sampled length in meters
    """
    return np.random.choice(reference_lengths)


def generate_calibrated_grid_network(width, height, target_spacing, reference_lengths, noise_level=10.0):
    """
    Generate grid network with calibrated segment lengths.
    
    Args:
        width: Width in meters
        height: Height in meters
        target_spacing: Base grid spacing
        reference_lengths: Segment length distribution from reference city
        noise_level: Random displacement amount
    
    Returns:
        NetworkX graph, positions dict
    """
    G = nx.Graph()
    
    # Create grid nodes
    node_id = 0
    node_positions = {}
    
    for x in range(0, width + 1, target_spacing):
        for y in range(0, height + 1, target_spacing):
            # Add noise to position
            nx_pos = x + np.random.uniform(-noise_level, noise_level)
            ny_pos = y + np.random.uniform(-noise_level, noise_level)
            
            # Clamp to window
            nx_pos = max(0, min(width, nx_pos))
            ny_pos = max(0, min(height, ny_pos))
            
            G.add_node(node_id, x=nx_pos, y=ny_pos)
            node_positions[node_id] = (nx_pos, ny_pos)
            node_id += 1
    
    # Create edges with lengths from reference distribution
    nodes_per_row = (height // target_spacing) + 1
    
    for node in list(G.nodes()):
        x, y = G.nodes[node]['x'], G.nodes[node]['y']
        
        # Right neighbor
        neighbor = node + nodes_per_row
        if neighbor in G.nodes():
            x2, y2 = G.nodes[neighbor]['x'], G.nodes[neighbor]['y']
            length = np.sqrt((x2 - x)**2 + (y2 - y)**2)
            if length >= MIN_SEGMENT_LENGTH:
                G.add_edge(node, neighbor, length=length)
        
        # Up neighbor
        neighbor = node + 1
        if neighbor in G.nodes():
            x2, y2 = G.nodes[neighbor]['x'], G.nodes[neighbor]['y']
            length = np.sqrt((x2 - x)**2 + (y2 - y)**2)
            if length >= MIN_SEGMENT_LENGTH:
                G.add_edge(node, neighbor, length=length)
    
    return G, node_positions


def randomly_remove_edges(G, removal_rate=0.2):
    """
    Randomly remove edges to create organic irregularity.
    
    Args:
        G: NetworkX graph
        removal_rate: Fraction of edges to remove (0-1)
    
    Returns:
        Modified graph
    """
    edges = list(G.edges())
    if len(edges) == 0:
        return G
    
    num_to_remove = int(len(edges) * removal_rate)
    
    if num_to_remove > 0:
        edges_to_remove = np.random.choice(len(edges), num_to_remove, replace=False)
        
        for idx in edges_to_remove:
            u, v = edges[idx]
            if G.has_edge(u, v):
                G.remove_edge(u, v)
    
    # Remove isolated nodes
    isolated = list(nx.isolates(G))
    G.remove_nodes_from(isolated)
    
    return G


print("✓ Network generation functions defined")

## Generate 20 Networks

In [ ]:
# Use London as reference for segment lengths
reference_city = 'london'
reference_lengths = reference_data[reference_city]['morphology']['segment_lengths']

print(f"Using {reference_city.upper()} segment length distribution")
print(f"Reference has {len(reference_lengths)} segments")
print(f"Length range: {min(reference_lengths):.1f}m - {max(reference_lengths):.1f}m")
print(f"Mean length: {np.mean(reference_lengths):.1f}m\n")

# Generate 20 networks
generated_networks = []

print("Generating networks...")
print("="*70)

for i in range(NUM_NETWORKS_TO_GENERATE):
    # Random parameters for variety
    spacing = np.random.randint(40, 70)
    noise_level = np.random.uniform(5, 15)
    removal_rate = np.random.uniform(0.1, 0.25)
    
    # Generate network
    G, pos = generate_calibrated_grid_network(
        WINDOW_SIZE_M, 
        WINDOW_SIZE_M, 
        spacing, 
        reference_lengths, 
        noise_level=noise_level
    )
    
    # Remove edges
    G = randomly_remove_edges(G, removal_rate=removal_rate)
    
    # Update positions after node removal
    pos = {n: (G.nodes[n]['x'], G.nodes[n]['y']) for n in G.nodes()}
    
    generated_networks.append({
        'id': i,
        'graph': G,
        'pos': pos,
        'params': {
            'spacing': spacing,
            'noise': noise_level,
            'removal': removal_rate
        }
    })
    
    print(f"Network {i+1:2d}: {G.number_of_nodes():3d} nodes, {G.number_of_edges():3d} edges ")

print("="*70)
print(f"\n✓ Generated {NUM_NETWORKS_TO_GENERATE} networks")

## Visualize All 20 Networks in One Grid

In [ ]:
# Create 4×5 grid
fig, axes = plt.subplots(4, 5, figsize=(20, 16))
axes = axes.flatten()

for idx, network_data in enumerate(generated_networks):
    ax = axes[idx]
    G = network_data['graph']
    pos = network_data['pos']
    
    # Window boundary
    ax.add_patch(Rectangle((0, 0), WINDOW_SIZE_M, WINDOW_SIZE_M,
                           fill=False, edgecolor='black', linestyle='-', linewidth=1))
    
    # Draw edges
    for u, v in G.edges():
        x = [pos[u][0], pos[v][0]]
        y = [pos[u][1], pos[v][1]]
        ax.plot(x, y, color='steelblue', linewidth=0.8, alpha=0.7, zorder=1)
    
    # Draw nodes
    for node in G.nodes():
        ax.scatter(pos[node][0], pos[node][1], s=8, c='darkblue',
                  zorder=2, alpha=0.6)
    
    ax.set_xlim(-10, WINDOW_SIZE_M + 10)
    ax.set_ylim(-10, WINDOW_SIZE_M + 10)
    ax.set_aspect('equal')
    ax.set_title(f'Network {idx+1}\n{G.number_of_nodes()}n, {G.number_of_edges()}e',
                fontsize=9, fontweight='bold')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)

plt.suptitle(f'Generated Pedestrian Networks (20 variations)\nCalibrated to {reference_city.upper()} segment length distribution',
            fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()

# Save as single SVG
plt.savefig('outputs/generated/visualizations/B1_all_20_networks.svg', format='svg', bbox_inches='tight', dpi=300)
print("Saved: outputs/generated/visualizations/B1_all_20_networks.svg")

plt.show()

## Compute Metrics for Generated Networks

In [ ]:
def calculate_bearing(p1, p2):
    """Calculate bearing (0-180°)."""
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    angle = math.atan2(dy, dx)
    bearing = math.degrees(angle)
    bearing = bearing % 180
    return bearing


def compute_morphology_metrics(G, pos):
    """Compute morphology metrics."""
    metrics = {}
    
    area_km2 = (WINDOW_SIZE_M / 1000.0) ** 2
    metrics['node_density'] = G.number_of_nodes() / area_km2
    
    degrees = [d for _, d in G.degree()]
    metrics['degree_distribution'] = dict(Counter(degrees))
    metrics['avg_degree'] = np.mean(degrees) if degrees else 0
    
    dead_ends = sum(1 for d in degrees if d == 1)
    metrics['dead_end_ratio'] = dead_ends / len(degrees) if degrees else 0
    
    lengths = []
    for u, v, data in G.edges(data=True):
        length = data.get('length', 0)
        lengths.append(length)
    
    metrics['segment_lengths'] = lengths
    metrics['avg_segment_length'] = np.mean(lengths) if lengths else 0
    
    bearings = []
    for u, v in G.edges():
        bearing = calculate_bearing(pos[u], pos[v])
        bearings.append(bearing)
    
    if bearings:
        counts, bins = np.histogram(bearings, bins=18, range=(0, 180))
        metrics['orientation_hist'] = (bins, counts)
    else:
        metrics['orientation_hist'] = (np.linspace(0, 180, 19), np.zeros(18))
    
    return metrics


# Compute metrics for all generated networks
print("Computing metrics for all generated networks...\n")

for network_data in generated_networks:
    metrics = compute_morphology_metrics(network_data['graph'], network_data['pos'])
    network_data['metrics'] = metrics

print("✓ Metrics computed for all networks")

## Summary Statistics

In [ ]:
# Aggregate statistics
all_node_counts = [net['graph'].number_of_nodes() for net in generated_networks]
all_edge_counts = [net['graph'].number_of_edges() for net in generated_networks]
all_avg_degrees = [net['metrics']['avg_degree'] for net in generated_networks]
all_avg_seg_lengths = [net['metrics']['avg_segment_length'] for net in generated_networks]

print("\n" + "="*70)
print("GENERATED NETWORKS SUMMARY (20 networks)")
print("="*70)
print(f"\nNodes:                {np.mean(all_node_counts):.1f} ± {np.std(all_node_counts):.1f}")
print(f"Edges:                {np.mean(all_edge_counts):.1f} ± {np.std(all_edge_counts):.1f}")
print(f"Avg Degree:           {np.mean(all_avg_degrees):.2f} ± {np.std(all_avg_degrees):.2f}")
print(f"Avg Segment Length:   {np.mean(all_avg_seg_lengths):.1f} ± {np.std(all_avg_seg_lengths):.1f}m")

# Compare to reference (adjusted)
ref_nodes = reference_data[reference_city]['graph'].number_of_nodes() // 2
ref_edges = reference_data[reference_city]['graph'].number_of_edges() // 2
ref_avg_degree = reference_data[reference_city]['morphology']['avg_degree'] / 2
ref_avg_seg_length = reference_data[reference_city]['morphology']['avg_segment_length']

print(f"\n{reference_city.upper()} REFERENCE (adjusted for doubled roads):")
print(f"Nodes:                {ref_nodes}")
print(f"Edges:                {ref_edges}")
print(f"Avg Degree:           {ref_avg_degree:.2f}")
print(f"Avg Segment Length:   {ref_avg_seg_length:.1f}m")
print("="*70)

## Save Generated Networks

In [ ]:
# Save all generated networks
with open('outputs/generated/networks/generated_networks_20.pkl', 'wb') as f:
    pickle.dump(generated_networks, f)

print("✓ Saved 20 generated networks to: outputs/generated/networks/generated_networks_20.pkl")
print(f"\nEach network includes:")
print(f"  - NetworkX graph")
print(f"  - Node positions")
print(f"  - Generation parameters")
print(f"  - Computed metrics")

## Next Steps

These 20 networks will be used for:

1. **Step 3**: Building footprint generation matching reference distributions
2. **Step 4**: Space syntax analysis and optimization
3. **Step 5**: Multi-objective calibration and selection
4. **Step 6**: Final validation and export